In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [ ]:
for i in housing:
  print(i)

data
target
frame
target_names
feature_names
DESCR


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train_full , x_test , y_train_full , y_test = train_test_split(housing.data , housing.target,random_state=42)

In [ ]:
x_train , x_valid , y_train , y_valid = train_test_split(x_train_full , y_train_full,random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:

class WideAndDeepModel(keras.Model):
  def __init__(self,units=30,activation="relu",**kwargs):
    super().__init__(**kwargs)
    self.norm_layer_wide = keras.layers.Normalization()
    self.norm_layer_deep = keras.layers.Normalization()
    self.hidden1 = keras.layers.Dense(units,activation=activation)
    self.hidden2 = keras.layers.Dense(units,activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)
  def call(self,inputs):
    input_wide , input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([norm_wide,hidden2])
    output = self.main_output(concat)
    return output

In [ ]:
 import tensorflow as tf

# Use tf.random.set_seed instead of tf.random_seed
tf.random.set_seed(42)
model = WideAndDeepModel(30, "relu", name="my_model")

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
 model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [ ]:
x_train_wide, x_train_deep = x_train[:, :5], x_train[:, 2:]
x_valid_wide, x_valid_deep = x_valid[:, :5], x_valid[:, 2:]
x_test_wide, x_test_deep = x_test[:, :5], x_test[:, 2:]

In [ ]:
model.norm_layer_deep.adapt(x_train_deep)
model.norm_layer_wide.adapt(x_train_wide)

In [ ]:
history = model.fit((x_train_wide, x_train_deep), y_train, epochs=10,
                    validation_data=((x_valid_wide, x_valid_deep), y_valid))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'my_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 1.4068 - loss: 2.0462 - val_RootMeanSquaredError: 0.7594 - val_loss: 0.5766
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7443 - loss: 0.5546 - val_RootMeanSquaredError: 0.6842 - val_loss: 0.4682
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6771 - loss: 0.4586 - val_RootMeanSquaredError: 0.6305 - val_loss: 0.3976
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - RootMeanSquaredError: 0.6502 - loss: 0.4230 - val_RootMeanSquaredError: 0.6093 - val_loss: 0.3712
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6317 - loss: 0.3992 - val_RootMeanSquaredError: 0.6572 - val_loss: 0.4320
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6213 - loss: 0.3863 - val_RootMeanSquaredError: 0.6328 - val_loss: 0.4005
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.6147 - loss: 0.3780 -

In [ ]:
eval_results = model.evaluate((x_test_wide, x_test_deep), y_test)
eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5943 - loss: 0.3532


[0.3460886776447296, 0.5882930159568787]

In [ ]:
x_new_wide, x_new_deep = x_test_wide[:3], x_test_deep[:3]
y_pred = model.predict((x_new_wide, x_new_deep))
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


array([[0.4847257],
       [1.6827755],
       [3.492555 ]], dtype=float32)

In [ ]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])